In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# PCA (not a sklearn logistic regression package)
# because dataset has 5000 features, use PCA(Principal Component Analysis) to reduce the number of features to 2 to form the graph

def load_data(file_path):
    data = pd.read_csv(file_path)
    X = data.iloc[:, 2:].values  # Features (column 3 to last column)
    y = data.iloc[:, 1].values   # Target variable (y is the second column)
    return X, y

def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

# LOSS FUNCTION
# RuntimeWarning: divide by zero encountered in log
#  return -np.mean(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
# when np.log takes in value of 0 (if y_hat = 0 or 1) 
# the logarithm of 0 is undefined in the real numbers, which will lead to this warning.

# RuntimeWarning: invalid value encountered in multiply
#  return -np.mean(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
# when np.log takes in negative value (y_hat < 0 or y_hat > 1)
# this will result in NaN (Not a Number) values in the calculation.
def loss(y, y_hat):
    eps = 1e-8  # Small epsilon to prevent log(0)
    y_hat = np.clip(y_hat, eps, 1 - eps)  # Clip y_hat to (eps, 1-eps) range
    return -np.mean(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))

def gradients(X, y, y_hat):
    m = X.shape[0]
    dw = (1/m) * np.dot(X.T, (y_hat - y))
    db = (1/m) * np.sum(y_hat - y)
    return dw, db

# NORMALISE FUNCTION
# RuntimeWarning: invalid value encountered in divide occurs when there are elements in X.std(axis=0) that are 0
# if all values in a column of X are same, will result in a standard deviation of 0
# create a boolean mask zero_std_mask to identify columns where the standard deviation is 0
# handle columns with zero standard deviation separately to avoid division by 0 (subtract the mean (X_mean) directly)
# for columns with non-zero standard deviation, normalise as usual
def normalise(X):
    # calculate mean and standard deviation
    X_mean = X.mean(axis=0)
    X_std = X.std(axis=0)
    
    # check for zero standard deviation
    zero_std_mask = X_std == 0
    
    # avoid division by zero and normalise
    X_normalised = np.zeros_like(X)
    X_normalised[:, ~zero_std_mask] = (X[:, ~zero_std_mask] - X_mean[~zero_std_mask]) / X_std[~zero_std_mask]
    X_normalised[:, zero_std_mask] = X[:, zero_std_mask] - X_mean[zero_std_mask]
    
    return X_normalised

def train(X, y, bs, epochs, lr):
    m, n = X.shape
    w = np.zeros((n, 1))
    b = 0
    y = y.reshape(m, 1)
    X = normalise(X)
    losses = []

    for epoch in range(epochs):
        for i in range((m-1) // bs + 1):
            start_i = i * bs
            end_i = start_i + bs
            xb = X[start_i:end_i]
            yb = y[start_i:end_i]

            y_hat = sigmoid(np.dot(xb, w) + b)
            dw, db = gradients(xb, yb, y_hat)
            w -= lr * dw
            b -= lr * db

        l = loss(y, sigmoid(np.dot(X, w) + b))
        losses.append(l)
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {l}')

    return w, b, losses

def predict(X, w, b):
    X = normalise(X)
    preds = sigmoid(np.dot(X, w) + b)
    return np.array([1 if i > 0.5 else 0 for i in preds])

def accuracy(y, y_hat):
    return np.sum(y == y_hat) / len(y)

def plot_decision_boundary(X, y, w, b):
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    
    x1 = [min(X_pca[:, 0]), max(X_pca[:, 0])]
    m = -w[0]/w[1]
    c = -b/w[1]
    x2 = m * np.array(x1) + c
    
    plt.figure(figsize=(10, 8))
    plt.plot(X_pca[:, 0][y == 0], X_pca[:, 1][y == 0], "g^")
    plt.plot(X_pca[:, 0][y == 1], X_pca[:, 1][y == 1], "bs")
    plt.xlim([min(X_pca[:, 0]) - 1, max(X_pca[:, 0]) + 1])
    plt.ylim([min(X_pca[:, 1]) - 1, max(X_pca[:, 1]) + 1])
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.title('Decision Boundary')
    plt.plot(x1, x2, 'y-')
    plt.show()

# load and process the dataset
file_path = "/kaggle/input/mlprojectdataset/train_tfidf_features.csv"
X, y = load_data(file_path)

# train the model
batch_size = 100
epochs = 1000
learning_rate = 0.01

w, b, losses = train(X, y, bs=batch_size, epochs=epochs, lr=learning_rate)
y_pred = predict(X, w, b)
acc = accuracy(y, y_pred)

print(f'Weights: {w}')
print(f'Bias: {b}')
print(f'Accuracy: {acc}')

# plot the decision boundary
plot_decision_boundary(X, y, w, b)